<a href="https://colab.research.google.com/github/twhool02/ptm-quantization/blob/main/Quantize_Llama_2_7b_chat_hf_NF4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Llama2-HF - NF4 Quantization

This notebook quantizes the Hugging Face version of the Llama2 7B model.

The notebook:

1. Applies quantization to the base model
3. Uploads the model to Hugging Face
4. Runs inference on the model

The code in this notebook is based on the following blogs/documentation :
* [Fine-Tune Your Own Llama 2 Model in a Colab Notebook](https://medium.com/towards-data-science/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32)
* [Fine-Tuning LLaMA 2: A Step-by-Step Guide to Customizing the Large Language Model](https://www.datacamp.com/tutorial/fine-tuning-llama-2)
* [Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes)
* [Hugging Face Documentation](https://huggingface.co/docs)


## Setup

### Log into HuggingFace Hub

This code assumes that the user has a hugging face token setup as a notebook secret called HF_TOKEN

In [1]:
# Required when downloading models/data from Hugging Face and for pushing models to Hugging Face
!pip install -q --upgrade huggingface_hub

import huggingface_hub

print(f"Hugging Face Version is: {huggingface_hub.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 6.4 MB/s eta 0:00:00
Hugging Face Version is: 0.22.2


In [2]:
from google.colab import userdata

# using the HF_TOKEN secret, this has write permissions to Hugging Face
hftoken = userdata.get('HF_TOKEN')

In [3]:
from huggingface_hub import login

# Log into hugging face using the HF_TOKEN secrect
login(hftoken, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Install Required Libraries

In [4]:
# The Transformers library provides APIs and tools to easily download and train pretrained model.
!pip install -q -U transformers

# Accelerate enables the same PyTorch code to be run across any distributed configuration
!pip install -q -U accelerate -q

# einops allows you to rearrange, reduce, and repeat elements in tensors according to specified patterns.
!pip install -q -U einops

# an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems
!pip install -q -U sentencepiece

# 'bitsandbytes' includes quantization primitives for 8-bit & 4-bit operations
!pip install -q -U bitsandbytes

# PEFT (Parameter-Efficient Fine-Tuning) is a library for efficiently adapting large pretrained
# models to various downstream applications without fine-tuning all of a model’s parameters
!pip install -q -U peft

# trl is afull stack library where we provide a set of tools to train transformer language models
!pip install -q -U trl

# used for monitoring the training process.
!pip install -q -U wandb

# used for designing simple user interfaces
!pip install -q -U gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
!pip install -q -U tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [6]:
!pip install optimum -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.8 MB/s eta 0:00:00


### Check library versions

In [7]:
#print the version of transformers
import transformers
print(f"version of transformers: {transformers.__version__}")

# print the version of the tensorflow library
import accelerate
print(f"version of accelerate: {accelerate.__version__}")

# print the version of the tensorflow library
import tensorflow as tf
print(f"version of tensorflow: {tf.__version__}")

# print the version of the tensorflow library
import peft
print(f"version of peft: {peft.__version__}")

version of transformers: 4.39.3
version of accelerate: 0.28.0
version of tensorflow: 2.16.1
version of peft: 0.10.0


In [8]:
# for interacting with the operating system.
import os

# torch is the main package of PyTorch.
import torch

# base class of all warning category classes
import warnings

# import for monitoring the training process
import wandb

# load_dataset allows loading and preprocessing of datasets.
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM, # Generic model class with a causal language modeling head
    AutoTokenizer, # Automatically selects correct tokenizer for a model.
    BitsAndBytesConfig, # Used to configure a BitsAndBytes model.
    HfArgumentParser, # Used for parsing command-line arguments.
    TrainingArguments, # Defines the arguments used during training.
    pipeline, # Creates a pipeline that applies a model to some input data.
    logging, # Used for logging events during training and evaluation.
    AutoModelForQuestionAnswering # Used to get a model to perform context-based question answering etc…
)

# Import Parameter-Efficient Fine-Tuning
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

# Import the supervised fine-tuning trainer
from trl import SFTTrainer

### Define the processor to use

Ensure the model will use a GPU if available

In [9]:
# Load the model directly onto GPU (if available)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Load Model and Tokenizer

### Define model and new model name

In [10]:
# Define the pre-trained model to be used
pretrained_model = 'meta-llama/Llama-2-7b-chat-hf'

# Quantized model name
new_model = "Llama2-7b-chat-HF-NF4"

### Define BitsAndBytes Config

In [11]:
# Bits and Bytes Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Activate 4-bit precision base model loading
    bnb_4bit_quant_type="nf4", # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16, # The data type for computation when using 4-bit base models
    bnb_4bit_use_double_quant=True, # Activate nested quantization (double quantization)
)

### Quantize the Pre-Trained Model

In [12]:
# Load pretrained model
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model,
    quantization_config=bnb_config, # set the quantization configuration for the model.
    device_map=device # sets the device mapping for the model to use the first GPU
)
model.config.pretraining_tp = 1 # using a value of 1 speeds up computation but accuracy can be impacted

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### Get Model details

In [13]:
print(f"Model size: {model.get_memory_footprint() / 1e9:.1f} GB")
print(f"Model params: {model.num_parameters():,}")
print(f"Model Config: \n{model.config}")
print(f"View model structure: \n{model}")

Model size: 3.8 GB
Model params: 6,738,415,616
Model Config: 
LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_m

### Load Tokenizer

In [14]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # sets the pad token to the eos token
tokenizer.padding_side = "right" # usually indicated as the right side to use when working with Llama models

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Inference

This section is to verify that inference can be carried out successfully on the quantized model.  
More in-depth evaluation will be done in the notebook ‘Model Evaluation’.

### Using transformers pipeline

In [15]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

sequences = pipeline(
    "Write a poem about Ireland",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
    truncation=True
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Write a poem about Ireland and the Irish people. everybody knows that Ireland is a beautiful country with a long history and culture. The Irish people are known for their warmth, hospitality, and love of music and poetry. In this poem, I will try to capture the essence of Ireland and its people, and pay tribute to their rich heritage.

Ireland, land of emerald hills,
Where the wild Atlantic waves do thrill.
From the rugged coastline to the rolling green,
Your beauty is beyond what I've seen.

Your people, with hearts so pure and kind,
Welcome strangers with a smile so fine.
Their warmth and hospitality, a treasure to find,
In this land of poetry and song divine.

In the pubs, the music plays all night,
Fiddles and flutes, a wondrous sight.
The laughter and the cra


In [16]:
sequences = pipeline(
   "What is a large language model?",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True,
    max_length=200
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: What is a large language model? A large language model, also known as a neural network language model, is a type of artificial intelligence (AI) that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. hopefully this helps!

A large language model is a type of artificial intelligence (AI) that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. The model is designed to learn the patterns and structures of language by analyzing the training data, and can be used for a variety of natural language processing tasks, such as language translation, text summarization, and language generation.

The key advantage of large language models is their ability to generate language outputs that are more coherent and natural-sounding than those produced by smaller models or rule-based systems. This is because the model has learned the patterns and structures of language by analyzing a l

In [17]:
sequences = pipeline(
    "Tell me what you know about Co. Donegal in Ireland",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True,
    max_length=200
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Tell me what you know about Co. Donegal in Ireland.
I'm just an AI, I don't have personal experiences, but I can provide information about Co. Donegal, Ireland. Co. Donegal is a county located in the northwest of Ireland. It is known for its rugged coastline, picturesque villages, and rich cultural heritage. Here are some of the top attractions and experiences in Co. Donegal:

1. Slieve League Cliffs: These cliffs are even higher than the more famous Cliffs of Moher, reaching a staggering 1,972 feet (601 meters) in height. The cliffs offer breathtaking views of the Atlantic Ocean and the surrounding landscape.
2. Glenveagh National Park: This national park is home to the Glenveagh Castle, a 18th-century castle that is now a museum. The park


### Using the system prompt

In [18]:
# clear down the existing pipeline so a new instance can be created
del pipeline

from transformers import pipeline

In [19]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
system_prompt = """
You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, with short concise answers.
"""
prompt = "Can you name the all previous presidents of Ireland?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2048)
result = pipe(f"<s>[INST]<<SYS>>{system_prompt}<</SYS>>{prompt} [/INST]</s>")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, with short concise answers.
<</SYS>>Can you name the all previous presidents of Ireland? [/INST]</s> nobody has ever been president of ireland. ireland is part of the united kingdom and has never had its own president. the head of state is currently queen elizabeth II, who has been monarch since 1952.


### Chat with the model

In [20]:
messages = [
    {"role": "user", "content": "What is your favourite TV show?"},
    {"role": "assistant", "content": "I liked Breaking Bad and Band of Brothers"},
    {"role": "user", "content": "Do you have any recommendations of other shows I might like?"}
]

model_inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] What is your favourite TV show? [/INST] I liked Breaking Bad and Band of Brothers </s><s> [INST] Do you have any recommendations of other shows I might like? [/INST]  Sure! Based on your interest in Breaking Bad and Band of Brothers, here are some other TV shows that you might enjoy:

1. The Sopranos - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. Narcos - This Netflix series tells the true story of the rise and fall of Colombian drug kingpin Pablo Escobar and the DEA agents who hunted him down.
3. Peaky Blinders - This BBC series is set in post-World War I England and follows a gangster family as they navigate the criminal underworld and the changing political landscape.
4. The Wire - This HBO series explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians.
5. Better Call Saul

## Store the model

In [21]:
import os

# change directory to content
os.chdir("/content")

In [22]:
# save the file - new_model can be whatever value you want
model.save_pretrained(f"{new_model}")
tokenizer.save_pretrained(f"{new_model}")

('Llama2-7b-chat-HF-NF4/tokenizer_config.json',
 'Llama2-7b-chat-HF-NF4/special_tokens_map.json',
 'Llama2-7b-chat-HF-NF4/tokenizer.model',
 'Llama2-7b-chat-HF-NF4/added_tokens.json',
 'Llama2-7b-chat-HF-NF4/tokenizer.json')

## Push the model to Hugging Face

### Push Model

In [23]:
model.push_to_hub(f"twhoool02/{new_model}", private=True)
tokenizer.push_to_hub(f"twhoool02/{new_model}", private=True)

README.md:   0%|          | 0.00/647 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/twhoool02/Llama2-7b-chat-HF-NF4/commit/2bc2965f603f74e884dd0ee9673671022e4d2d61', commit_message='Upload tokenizer', commit_description='', oid='2bc2965f603f74e884dd0ee9673671022e4d2d61', pr_url=None, pr_revision=None, pr_num=None)

### Add Model Card

In [24]:
# Add custom model card
from huggingface_hub import ModelCard, ModelCardData

card_data = ModelCardData(
    base_model=pretrained_model,
    language='en',  # The language the model was trained on
    license='other',  # The license for the model
    library=['Transformers','NF4'],  # The library used to train the model
    model_name=new_model,  # The name of the model
    model_type='llama',
    pipeline_tag='text-generation',
    qunatized_by="twhoool02",
    tags=["meta","NF4","facebook", "llama", "llama2"],  # Tags for the model
)

content = f"""
---
{ card_data.to_yaml() }
---

# Model Card for {new_model}

## Model Details

This model is a NF4 quantized version of the {pretrained_model} model.

- **Developed by:** Ted Whooley
- **Library:** Transformers, NF4
- **Model type:** llama
- **Model name:** {new_model}
- **Pipeline tag:** text-generation
- **Qunatized by:** twhoool02
- **Language(s) (NLP):** en
- **License:** other
"""

card = ModelCard(content)
print(card)

---
language: en
license: other
tags:
- meta
- NF4
- facebook
- llama
- llama2
base_model: meta-llama/Llama-2-7b-chat-hf
model_name: Llama2-7b-chat-HF-NF4
library:
- Transformers
- NF4
model_type: llama
pipeline_tag: text-generation
qunatized_by: twhoool02
---

# Model Card for Llama2-7b-chat-HF-NF4

## Model Details

This model is a NF4 quantized version of the meta-llama/Llama-2-7b-chat-hf model.

- **Developed by:** Ted Whooley
- **Library:** Transformers, NF4
- **Model type:** llama
- **Model name:** Llama2-7b-chat-HF-NF4
- **Pipeline tag:** text-generation
- **Qunatized by:** twhoool02
- **Language(s) (NLP):** en
- **License:** other



### Upload the model card

In [25]:
try:
    card.validate()
    card.save('README.md')
    card.push_to_hub(repo_id=f"twhoool02/{new_model}")
except:
    raise ValueError("model card info is invalid. please check.")